In [5]:
from google.colab import drive
import os

# 1️⃣ Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls

drive  sample_data


In [ ]:
import math
import inspect
from dataclasses import dataclass
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
vocab = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '+', '&', '*']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
padding_token_index = 13
end_token_index = 12

In [ ]:
# create a mapping from chars to ints
stoi = {ch:i for i, ch in enumerate(vocab)}
itos = {i:ch for i, ch in enumerate(vocab)}
encode = lambda s:[stoi[c] for c in s] # encoder: take a string, output a list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of ints, output a string

print(encode("1+2=3&"))
print(decode(encode("1+2=3&")))

[1, 11, 2, 10, 3, 12]
1+2=3&


In [ ]:
# train test split
train_set_1 = np.random.choice(np.arange(10), 8, replace=False)
train_set_2 = np.random.choice(np.arange(10, 100), 72, replace=False)
train_set_3 = np.random.choice(np.arange(100, 1000), 720, replace=False)
test_1 = np.setdiff1d(np.arange(10), train_set_1)
test_2 = np.setdiff1d(np.arange(10, 100), train_set_2)
test_3 = np.setdiff1d(np.arange(100, 1000), train_set_3)
test_12 = np.concatenate([test_1, test_2])
test = np.concatenate([test_1, test_2, test_3])
print(np.sort(train_set_1))
print(np.sort(train_set_2))
print(np.sort(test_1))
print(np.sort(test_2))
print(np.sort(test))

[0 3 4 5 6 7 8 9]
[10 11 12 13 15 16 17 18 19 20 22 23 24 25 27 30 31 32 34 35 36 37 38 39
 41 42 43 44 47 48 50 52 53 54 55 56 58 59 60 61 62 63 64 65 66 68 69 70
 71 72 73 74 75 77 79 80 81 82 83 85 86 88 90 91 92 93 94 95 96 97 98 99]
[1 2]
[14 21 26 28 29 33 40 45 46 49 51 57 67 76 78 84 87 89]
[  1   2  14  21  26  28  29  33  40  45  46  49  51  57  67  76  78  84
  87  89 108 109 113 116 122 128 131 134 136 137 142 144 146 148 157 163
 169 173 180 181 190 195 197 210 212 214 224 226 228 231 232 237 242 244
 259 261 266 275 277 283 284 288 289 294 305 315 317 328 331 334 337 339
 341 353 360 364 373 374 390 407 427 438 447 448 452 455 466 467 490 500
 502 503 504 505 509 511 519 529 536 538 544 550 551 555 556 558 566 573
 575 579 586 590 591 598 600 601 602 604 605 609 614 619 624 625 631 632
 635 636 655 663 670 671 674 680 681 682 684 690 694 700 706 709 738 746
 752 759 761 764 767 769 773 777 783 787 790 792 794 797 799 802 803 812
 813 822 824 826 834 837 840 842 852 856 85

In [ ]:
def get_batch(phase=None, batch_size=32, block_size=15, mode='train', train_set_1=train_set_1, train_set_2=train_set_2, train_set_3=train_set_3, test=test):

    def sample_from_arrays(one_digit_array, two_digit_array, three_digit_array,
                       phase):
      # stratified sampling
        total_num_in_set = 1000
        if phase == 1:
          sampled_one = np.random.choice(one_digit_array, int(total_num_in_set*1), replace=True)
          sampled_two = np.random.choice(two_digit_array, int(total_num_in_set*0), replace=True)
          sampled_three = np.random.choice(three_digit_array, int(total_num_in_set*0), replace=True)
        elif phase == 2:
          sampled_one = np.random.choice(one_digit_array, int(total_num_in_set*0.3), replace=True)
          sampled_two = np.random.choice(two_digit_array, int(total_num_in_set*0.7), replace=True)
          sampled_three = np.random.choice(three_digit_array, int(total_num_in_set*0), replace=True)
        elif phase == 3:
          sampled_one = np.random.choice(one_digit_array, int(total_num_in_set*0.15), replace=True)
          sampled_two = np.random.choice(two_digit_array, int(total_num_in_set*0.2), replace=True)
          sampled_three = np.random.choice(three_digit_array, int(total_num_in_set*0.65), replace=True)
        # Combine all sampled numbers into one dataset
        new_dataset = np.concatenate([sampled_one, sampled_two, sampled_three])

        return new_dataset


    if mode == 'train':
      sampled_set = sample_from_arrays(train_set_1, train_set_2, train_set_3, phase)
      # random choose a and b from set
      a = np.random.choice(sampled_set, batch_size)
      b = np.random.choice(sampled_set, batch_size)
      c = a + b
    else:
      a = np.random.choice(test, batch_size)
      b = np.random.choice(test, batch_size)
      c = a + b

    x_list, y_list = [], []
    for i, j, k in zip(a, b, c):
        # construct X: "i+j=k&"
        x_str = f"{i}+{j}={k}&"
        # print(x_str)
        x_encoded = encode(x_str)
        x_padded = x_encoded + [padding_token_index] * (block_size - len(x_encoded))
        x_list.append(torch.tensor(x_padded, dtype=torch.int64))

        # construct Y: "k&"
        y_encoded = encode(x_str)[1:]
        y_encoded.append(end_token_index)
        y_padded = y_encoded + [padding_token_index] * (block_size - len(y_encoded))
        y_list.append(torch.tensor(y_padded, dtype=torch.int64))

    x_tensor = torch.stack(x_list).to(device)
    y_tensor = torch.stack(y_list).to(device)
    return x_tensor, y_tensor

In [ ]:
get_batch(3)

(tensor([[ 4,  1,  3, 11,  4,  8,  5, 10,  8,  9,  8, 12, 13, 13, 13],
         [ 4,  5,  4, 11,  7,  7,  5, 10,  1,  2,  2,  9, 12, 13, 13],
         [ 6, 11,  3,  8,  8, 10,  3,  9,  4, 12, 13, 13, 13, 13, 13],
         [ 1,  8,  6, 11,  9,  8, 10,  2,  8,  4, 12, 13, 13, 13, 13],
         [ 2,  0,  2, 11,  6,  5, 10,  2,  6,  7, 12, 13, 13, 13, 13],
         [ 5,  0,  8, 11,  7,  6,  0, 10,  1,  2,  6,  8, 12, 13, 13],
         [ 6,  0, 11,  1,  2,  4, 10,  1,  8,  4, 12, 13, 13, 13, 13],
         [ 5,  6,  3, 11,  4,  9,  6, 10,  1,  0,  5,  9, 12, 13, 13],
         [ 6,  4,  8, 11,  3, 10,  6,  5,  1, 12, 13, 13, 13, 13, 13],
         [ 2,  2,  9, 11,  3,  2, 10,  2,  6,  1, 12, 13, 13, 13, 13],
         [ 6,  4,  9, 11,  1,  2,  4, 10,  7,  7,  3, 12, 13, 13, 13],
         [ 9,  1,  6, 11,  9,  8,  8, 10,  1,  9,  0,  4, 12, 13, 13],
         [ 1,  9,  2, 11,  6,  0, 10,  2,  5,  2, 12, 13, 13, 13, 13],
         [ 1,  0, 11,  2,  9,  9, 10,  3,  0,  9, 12, 13, 13, 13, 13],
      

In [ ]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias=True): # class constructor
        super().__init__()
        # nn.Parameter, pytorch optimize will update the value of this parameter during training
        self.weight = nn.Parameter(torch.ones(ndim)) # trainable parameter
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None # trainable parameter

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        assert n_embd % n_head == 0, "Embedding dimension must be divisible by the number of heads."

        # Store hyperparameters
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        self.block_size = block_size

        # Key, Query, Value projections
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # Output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # Regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

        # Check for Flash Attention availability
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # Causal mask for slow attention
            self.register_buffer(
                "bias",
                torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
            )

    def forward(self, x):
        B, T, C = x.size()  # Batch size, sequence length, embedding dimension

        # Compute Q, K, V
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)  # Split into Q, K, V (B, T, n_embd)

        # Reshape for multi-head attention
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)

        # Flash Attention or fallback to manual implementation
        if self.flash:
            y = torch.nn.functional.scaled_dot_product_attention(
                q, k, v,
                attn_mask=None,
                dropout_p=self.dropout if self.training else 0,
                is_causal=True
            )
        else:
            # Manual attention with causal masking
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))  # Scaled dot product
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))  # Apply causal mask
            att = F.softmax(att, dim=-1)  # Normalize attention scores
            att = self.attn_dropout(att)
            y = att @ v  # Apply attention weights to values (B, n_head, T, head_size)

        # Reshape back to original format
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # Reassemble heads

        # Output projection and residual dropout
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module): # FFN

    def __init__(self, n_embd, dropout, bias=True):
        super().__init__()
        self.c_fc    = nn.Linear(n_embd, 4 * n_embd, bias=bias)
        self.gelu    = nn.GELU() # nonlinear activation function
        self.c_proj  = nn.Linear(4 * n_embd, n_embd, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        # LayerNorm and CausalSelfAttention with explicit parameters
        self.ln_1 = LayerNorm(n_embd, bias=bias)
        self.attn = CausalSelfAttention(n_embd, n_head, dropout, block_size, bias=bias)
        self.ln_2 = LayerNorm(n_embd, bias=bias)
        self.mlp = MLP(n_embd, dropout, bias=bias)  # MLP with explicit parameters

    def forward(self, x):
        # Apply residual connection and pre-normalization
        x = x + self.attn(self.ln_1(x))  # Apply LayerNorm before attention
        x = x + self.mlp(self.ln_2(x))  # Apply LayerNorm before MLP
        return x


class GPT(nn.Module):

    def __init__(self, vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=True):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        super().__init__()
        assert vocab_size is not None
        assert block_size is not None
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.dropout = dropout
        self.bias = bias

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, n_embd), # token embeddings
            wpe = nn.Embedding(block_size, n_embd), # positional embeddings
            drop = nn.Dropout(dropout),
            h = nn.ModuleList([Block(n_embd, n_head, dropout, block_size, bias=bias) for _ in range(n_layer)]), # a stack of n_layer blocks
            ln_f = LayerNorm(n_embd, bias=bias), # final layer norm
        ))
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False) # projects the final transformer output to the vocab size

        # init all weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.cblock_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        logits = self.lm_head(x)

        loss = None

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=13)
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            # loss = None

        return logits, loss

In [ ]:
eval_iters = 200

@torch.no_grad()
def estimate_loss(phase, models):
    out = {}
    models.eval()
    for split in ['train', 'val']:
      losses = torch.zeros(eval_iters)
      for k in range(eval_iters):
          X, Y = get_batch(phase, mode=split)
          padding_mask_x = (X != padding_token_index).long()
          logits, loss = models(X, Y)
          losses[k] = loss.item()
      out[split] = losses.mean()
    models.train()
    return out

In [ ]:
# batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 15 # what is the maximum context length for predictions?
max_iters = 50000
# num_epochs = 100
eval_interval = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20
n_embd = 1024
n_head = 64
n_layer = 8
dropout = 0.0
# # torch.manual_seed(1337)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(1337)
bias = True # if using bias inside all Linear layers
vocab_size = len(vocab)

In [ ]:
def accuracy(model, acc_sample):
    correct = 0
    for j in range(100):
        if acc_sample == "all":
            a = np.random.choice(np.arange(1000), 1)
            b = np.random.choice(np.arange(1000), 1)
        elif acc_sample == "test":
            a = np.random.choice(test, 1)
            b = np.random.choice(test, 1)

        c = a + b
        input = f"{a.item()}+{b.item()}="
        context = torch.tensor(encode(input), dtype=torch.long, device=device)
        output = generate(model, context, 100, 1)
        if output == f"{a.item()}+{b.item()}={c.item()}":
            correct += 1
    print(f"Accuracy for addition in {acc_sample}: {correct / 100} ")
    return correct / 100

In [ ]:
@torch.no_grad()
def generate(model, idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Generate a sequence of tokens given an initial sequence.

    Parameters:
        model (nn.Module): The model used for generation.
        idx (torch.Tensor or list): Initial sequence of indices (LongTensor of shape (b,t)).
        max_new_tokens (int): Number of new tokens to generate.
        temperature (float): Scaling factor for logits before softmax.
        top_k (int, optional): If specified, restricts sampling to top k tokens.

    Returns:
        torch.Tensor: The generated sequence.
    """
    idx = idx.unsqueeze(0) if idx.dim() == 1 else idx
    idx = torch.tensor(idx, device=model.device) if not isinstance(idx, torch.Tensor) else idx.to(model.device)

    for _ in range(max_new_tokens):
        # Ensure context length does not exceed model's block size
        idx_cond = idx if idx.size(1) <= model.block_size else idx[:, -model.block_size:]

        # Forward pass to get logits
        logits, _ = model(idx_cond)

        # Extract logits for the last token and apply temperature scaling
        logits = logits[:, -1, :] / temperature

        # Apply top-k filtering if necessary
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')

        # Convert logits to probabilities
        probs = F.softmax(logits, dim=-1)

        # Sample next token
        idx_next = torch.multinomial(probs, num_samples=1)

        if idx_next == end_token_index:
            break
        # Append sampled token to sequence

        # Append sampled token to sequence
        idx = torch.cat((idx, idx_next), dim=1)

    return decode(idx.tolist()[0])


In [ ]:
model = GPT(vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=bias)
m = model.to(device)

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [ ]:
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

phase = 1
best_acc_all = 0
best_acc_test = 0
counter = 0
patience = 20
best_loss = float('inf')

for iter in tqdm(range(max_iters), desc="Training Progress"):
    if iter > 1000:
      phase = 2
    if iter > 5000:
      phase = 3

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(phase, model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if iter > 5000:
            acc_all = accuracy(model, "all")
            acc_test = accuracy(model, "test")
            if acc_all > best_acc_all or acc_test > best_acc_test or losses['val'] < best_loss:
                best_acc_all = max(best_acc_all, acc_all)
                best_acc_test = max(best_acc_test, acc_test)
                best_loss = min(best_loss, losses['val'])
            else:
              counter += 1
              if counter >= patience:
                print(f"Early Stopping at iteration {iter}")
                break

    # sample a batch of data

    xb, yb = get_batch(phase)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



100.815872 M parameters


Training Progress:   0%|          | 5/50000 [00:00<1:19:10, 10.53it/s]

step 0: train loss 3.2047, val loss 2.9880


Training Progress:   0%|          | 105/50000 [00:04<43:07, 19.29it/s]

step 100: train loss 0.3378, val loss 4.9608


...

Accuracy for addition in test: 0.84 


Training Progress:  23%|██▎       | 11300/50000 [10:24<20:14, 31.85it/s]

step 11300: train loss 0.9703, val loss 1.0744
Accuracy for addition in all: 0.87 


Training Progress:  23%|██▎       | 11300/50000 [10:28<35:54, 17.97it/s]

Accuracy for addition in test: 0.79 
Early Stopping at iteration 11300


In [ ]:
test_set = {1: test_1,
            2: test_12,
            3: test
            }

In [ ]:
def accuracy_print(model, num_digits, set_type):
        correct = 0
        if set_type == "all":
          for j in range(100):
              a = np.random.choice(np.arange(10**num_digits), 1)
              b = np.random.choice(np.arange(10**num_digits), 1)
              c = a + b
              input = f"{a.item()}+{b.item()}="
              context = torch.tensor(encode(input), dtype=torch.long, device=device)
              output = generate(model, context, 100, 1)
              if j // 10 == 0:
                  print(f"Input: {input}")
                  print(f"Output: {output}")
              if output == f"{a.item()}+{b.item()}={c.item()}":
                  correct += 1
        elif set_type == "test":
            for j in range(100):
                a = np.random.choice(test_set[num_digits], 1)
                b = np.random.choice(test_set[num_digits], 1)
                c = a + b
                input = f"{a.item()}+{b.item()}="
                context = torch.tensor(encode(input), dtype=torch.long, device=device)
                output = generate(model, context, 100, 1)
                if j // 10 == 0:
                    print(f"Input: {input}")
                    print(f"Output: {output}")
                if output == f"{a.item()}+{b.item()}={c.item()}":
                    correct += 1
        print(f"Accuracy for addition: {correct / 100} ")

In [ ]:
accuracy_print(model, 1, "all")

Input: 6+5=
Output: 6+5=11
Input: 5+4=
Output: 5+4=9
Input: 0+1=
Output: 0+1=11
Input: 9+8=
Output: 9+8=17
Input: 9+7=
Output: 9+7=16
Input: 4+6=
Output: 4+6=10
Input: 1+2=
Output: 1+2=3
Input: 3+7=
Output: 3+7=10
Input: 5+4=
Output: 5+4=9
Input: 6+8=
Output: 6+8=14
Accuracy for addition: 0.81 


In [ ]:
accuracy_print(model, 1, "test")

Input: 1+1=
Output: 1+1=
Input: 2+2=
Output: 2+2=8
Input: 2+1=
Output: 2+1=
Input: 2+1=
Output: 2+1=
Input: 1+1=
Output: 1+1=
Input: 1+1=
Output: 1+1=
Input: 2+1=
Output: 2+1=
Input: 2+1=
Output: 2+1=
Input: 1+1=
Output: 1+1=1
Input: 1+2=
Output: 1+2=3
Accuracy for addition: 0.14 


In [ ]:
accuracy_print(model, 2, "all")

Input: 75+76=
Output: 75+76=151
Input: 67+8=
Output: 67+8=75
Input: 1+68=
Output: 1+68=72
Input: 78+35=
Output: 78+35=113
Input: 35+53=
Output: 35+53=88
Input: 80+79=
Output: 80+79=159
Input: 38+4=
Output: 38+4=42
Input: 38+56=
Output: 38+56=93
Input: 45+95=
Output: 45+95=140
Input: 68+2=
Output: 68+2=70
Accuracy for addition: 0.92 


In [ ]:
accuracy_print(model, 2, "test")

Input: 28+89=
Output: 28+89=116
Input: 28+29=
Output: 28+29=57
Input: 29+1=
Output: 29+1=20
Input: 2+26=
Output: 2+26=28
Input: 46+26=
Output: 46+26=72
Input: 33+29=
Output: 33+29=62
Input: 26+49=
Output: 26+49=75
Input: 40+33=
Output: 40+33=73
Input: 78+87=
Output: 78+87=165
Input: 49+87=
Output: 49+87=136
Accuracy for addition: 0.83 


In [ ]:
accuracy_print(model, 3, "all")

Input: 836+723=
Output: 836+723=1559
Input: 640+226=
Output: 640+226=866
Input: 748+81=
Output: 748+81=829
Input: 396+664=
Output: 396+664=1060
Input: 580+857=
Output: 580+857=1437
Input: 519+626=
Output: 519+626=1155
Input: 136+729=
Output: 136+729=865
Input: 145+883=
Output: 145+883=1028
Input: 995+234=
Output: 995+234=1229
Input: 42+773=
Output: 42+773=815
Accuracy for addition: 0.87 


In [ ]:
accuracy_print(model, 3, "test")

Input: 519+84=
Output: 519+84=603
Input: 2+601=
Output: 2+601=207
Input: 812+315=
Output: 812+315=1137
Input: 999+590=
Output: 999+590=1589
Input: 690+294=
Output: 690+294=984
Input: 996+822=
Output: 996+822=1818
Input: 706+360=
Output: 706+360=1066
Input: 500+374=
Output: 500+374=874
Input: 709+865=
Output: 709+865=1574
Input: 614+353=
Output: 614+353=967
Accuracy for addition: 0.81 


In [ ]:
import subprocess

os.system('git config --global user.email "zifeibai@umich.edu"')
os.system('git config --global user.name "ZifeiBai"')

# 2️⃣ **Use Google Drive to store GitHub Token**
GITHUB_TOKEN_PATH = "/content/drive/MyDrive/URPS/github_token.txt"
if os.path.exists(GITHUB_TOKEN_PATH):
    with open(GITHUB_TOKEN_PATH, "r") as f:
        os.environ["GITHUB_TOKEN"] = f.read().strip()
else:
    print("❌ GitHub Token")
    exit(1)

# 3️⃣ **Set up GitHub remote repo**
GIT_PATH = "/content/drive/MyDrive/URPS/Git"
REPO_URL = f"https://{os.environ['GITHUB_TOKEN']}@github.com/ZifeiBai/URPS.git"

if not os.path.exists(GIT_PATH):
    print(f"📁 Creating directory: {GIT_PATH}")
    os.makedirs(GIT_PATH)

# 4️⃣ **If .git/ does not exsit， need to clone**
if not os.path.exists(os.path.join(GIT_PATH, ".git")):
    print("❌ Git repository not found. Cloning...")
    subprocess.run(f"rm -rf {GIT_PATH}", shell=True, check=True)
    subprocess.run(f"git clone {REPO_URL} {GIT_PATH}", shell=True, check=True)

# 5️⃣ **Enter Git repo**
os.chdir(GIT_PATH)
print("📂 Changed working directory to:", os.getcwd())


# 6️⃣ **Check Git status**
status_output = subprocess.run("git status", shell=True, capture_output=True, text=True)
print(status_output.stdout)

#  **Push to Git**
print("🚀 Adding files to Git...")
subprocess.run("git add .", shell=True, check=True)

print("📝 Committing changes...")
commit_output = subprocess.run('git commit -m "Auto update from Google Colab 2.6"', shell=True, capture_output=True, text=True)
print(commit_output.stdout)



print("📤 Pushing to GitHub...")
push_output = subprocess.run("git push origin main", shell=True, capture_output=True, text=True)
if "fatal" in push_output.stderr or "error:" in push_output.stderr:
    print("❌ Real Git Push Error:", push_output.stderr)
else:
    print("✅ Git Push Success!")

📂 Changed working directory to: /content/drive/MyDrive/URPS/Git
